## Check List 7.1. Demo - White board sweeping
* In this demo file, we will use two robots (Indy7, Panda) to sweep a white board, removing obstacles

* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

### Prepare the task scene
* Prepare Indy7 and panda, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [ ]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.utils.code_scraps import add_indy_sweep_tool, use_current_place_point_only, use_current_sub_binders_only,  \
                                                    finish_L_shape, set_l_shape_object, double_sweep_motions

## init combined robot config

In [ ]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

In [ ]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

## create scene builder

In [ ]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [ ]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
for rname in xyz_rpy_robots.keys():
    xyz_rpy = xyz_rpy_robots[rname]
    xyz_rpy_robots[rname] = (xyz_rpy[0], (0,0,xyz_rpy[1][2])) # set robot direction right vertical
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

## init planning pipeline

In [ ]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [ ]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
for rname, xyz_rpy in xyz_rpy_robots.items():
    gscene.create_safe(
        GEOTYPE.BOX, rname+"_base_col", "base_link", 
            dims=(0.5,0.3,0.02), center=xyz_rpy[0], rpy=xyz_rpy[1], 
            color=(0.8,0.8,0.8,0.8), display=True, fixed=True, collision=True)
# gscene.NAME_DICT["floor"].dims = (2,1,0.01)
gscene.add_virtual_guardrail(gscene.NAME_DICT["floor"])
gscene.set_workspace_boundary( -1, 1, -0.5, 0.5, -0.1, 1.05)

## add indy tool

In [ ]:
add_indy_sweep_tool(gscene, "indy0", face_name="brush_face")

## add targets

In [ ]:
gtem_dict = s_builder.detect_and_register(item_names=["target1", "target2", "target3", "target4", "target5"])

# add boxes

In [ ]:
gtem_dict.update(s_builder.detect_and_register(item_names=["box1", "box2", "box3", "long_1", "long_2"]))

In [ ]:
target_dict = {}
for gname, gtem in gtem_dict.items():
    if "target" in gname:
        target_dict[gname] = gtem

In [ ]:
gscene.update_markers_all()

## Register binders

In [ ]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot

In [ ]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,gscene.NAME_DICT['brush_face'].dims[2]/2), 
                     rpy=(0,np.pi,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)

## add objects

In [ ]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [ ]:
obj_list = []
for gname in sorted(gtem_dict.keys()):
    if "box" in gname or "long" in gname:
        obj_list.append(pscene.create_subject(oname=gname, gname=gname, _type=BoxObject, hexahedral=True, CLEARANCE=1e-3))

### planners

In [ ]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [ ]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
checkers_all = [rcheck, gcheck]
# tcheck = TaskClearanceChecker(pscene, gcheck)
# checkers_all = [tcheck, rcheck, gcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [ ]:
mplan.motion_filters = checkers_all

In [ ]:
gscene.show_pose(crob.home_pose)

In [ ]:
from pkg.utils.code_scraps import move_objects_down_until_collision, move_objects_up_until_no_collision

In [ ]:
for obj in obj_list:
    obj.geometry.collision = False
for gtem in target_dict.values():
    gtem.collision = False
    
move_objects_down_until_collision(target_dict.values(), gcheck, crob.home_dict)
move_objects_up_until_no_collision(target_dict.values(), gcheck, crob.home_dict)

for obj in obj_list:
    obj.geometry.collision = True
for gtem in target_dict.values():
    gtem.collision = True
    
move_objects_down_until_collision(obj_list, gcheck, crob.home_dict)
move_objects_up_until_no_collision(obj_list, gcheck, crob.home_dict)

In [ ]:
gscene.update_markers_all()

In [ ]:
for gname, gtem in target_dict.items():
    gtem_col = gscene.create_safe(
        GEOTYPE.BOX, gname+"_col", gtem.link_name, 
        dims=(0.12,0.23,gtem.dims[2]), center=(0,-0.055,0), rpy=(0,0,0), 
        color=(0.8,0.8,0.8,0.8), display=True, fixed=gtem.fixed, collision=True, parent=gname)

In [ ]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [ ]:
sweep_list = []
for gname in sorted(target_dict.keys()):
    gtem = target_dict[gname]
    sweep_ = pscene.create_subject(oname="sweep_{}".format(gname), gname=gname, _type=SweepLineTask, 
                                   action_points_dict = {gname+"_1": SweepFrame(gname+"_1", gtem, [0,0,gtem.dims[2]/2], [0,0,0]),
                                                       gname+"_2": SweepFrame(gname+"_2", gtem, [0,0,gtem.dims[2]/2], [0,0,0])})
    sweep_.fix_direction = False
    sweep_list.append(sweep_)

## Set initial condition

In [ ]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

# remove place points except for the current one
use_current_place_point_only(pscene, initial_state)
use_current_sub_binders_only(pscene, initial_state)

In [ ]:
pscene.subject_name_list

# Node Sampler

In [ ]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

# CustomRule

In [ ]:
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
tplan.custom_rule = SweepEntranceControlRule(pscene)

# Switcher for force logging

In [ ]:
import requests
from bs4 import BeautifulSoup
from pkg.utils.code_scraps import start_force_mode, stop_force_mode
import matplotlib.pyplot as plt

def print_min_max_force(ip_addr, JOINT_DOF, UI_PORT=9990, DT=1.0/2e3):
    uri = "http://{ip_addr}:{UI_PORT}/download_log".format(ip_addr=ip_addr, UI_PORT=UI_PORT)
    print(uri)
    log_dat = requests.get(uri)
    dat = log_dat.text
    lines = dat.split("\n")
    heads = lines[0].split(",")[:-1]
    data_mat = []
    for line in lines[1:]:
        data_line = list(map(float, line.split(",")[:-1]))
        if len(data_line)>0:
            data_mat.append(data_line)
    data_mat = np.array(data_mat)
    Fext = data_mat[:,JOINT_DOF*5:JOINT_DOF*6]
    Fext = Fext[-int(15.0/DT):]
    idx_peak = np.argmax(Fext[:,2])
    print("peak: {}".format(np.round(Fext[idx_peak,2], 1)))
    Fext = Fext[idx_peak+int(1.0/DT):idx_peak+int(4.0/DT), 2]
    plt.plot(Fext)
    print("force min/max: {} / {} in {}".format(np.round(np.min(Fext), 1), np.round(np.max(Fext), 1), len(Fext)))
    
class ModeSwitcherForceLog:
    def __init__(self, pscene, print_force=True, DT=1.0/2e3):
        self.pscene = pscene
        self.crob = pscene.combined_robot
        self.switch_delay = 0.5
        self.DT = DT
        self.print_force = print_force

    def switch_in(self, snode_pre, snode_new):
        switch_state = False
        for n1, n2 in zip(snode_pre.state.node, snode_new.state.node):
            if n1 == 1 and n2 == 2:
                switch_state = True
                break
        if switch_state:
            indy = self.crob.robot_dict['indy0']
            if indy is not None:
                start_force_mode(indy, switch_delay=self.switch_delay)
        return switch_state

    def switch_out(self, switch_state, snode_new):
        if switch_state:
            indy = self.crob.robot_dict['indy0']
            if indy is not None:
                if self.print_force:
                    sleep(4)
                stop_force_mode(indy, Qref=snode_new.traj[-1][self.crob.idx_dict['indy0']],
                                                              switch_delay=self.switch_delay)
                if self.print_force:
                    sleep(self.switch_delay)
                    print_min_max_force(indy.server_ip, len(crob.idx_dict["indy0"]), DT=self.DT)

In [ ]:
msfl = ModeSwitcherForceLog(pscene, print_force=False)

## Plan & run

In [ ]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
from pkg.utils.traj_utils import simplify_schedule, mix_schedule
mplan.reset_log(False)
gtimer.reset(scale=1, timeunit="s")
tplan.prepare()
mplan.update_gscene()

In [ ]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
from pkg.planning.constraint.constraint_common import get_binding_margins, fit_binding
crob.reset_connection(True,True)

In [ ]:
time.sleep(1)

In [ ]:
crob.grasp(True, True)
time.sleep(2)
crob.grasp(False, False)

In [ ]:
crob.joint_move_make_sure(crob.home_pose)

In [ ]:
obj_num = len(obj_list)
sweep_num = len(sweep_list)
from_state = initial_state
snode_schedule_all = []

In [ ]:
sweep_idx = 0
sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
goal_nodes = [("floor",)*obj_num+sweep_goal]
if sweep_idx < sweep_num-1:
    for i_s in range(obj_num):
        obj_goal = ["floor"]*obj_num
        obj_goal[i_s] = "grip1"
        goal_nodes += [tuple(obj_goal)+sweep_goal]
with gtimer.block("plan{}".format(sweep_idx), stack=True):
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=50, multiprocess=True, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
snode_schedule = mix_schedule(mplan, snode_schedule_safe)
from_state = snode_schedule[-1].state
snode_schedule_all.append(snode_schedule)
with gtimer.block("exe{}".format(sweep_idx), stack=True):
    ppline.execute_schedule(snode_schedule, mode_switcher=msfl)

In [ ]:
sweep_idx = 1
sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
goal_nodes = [("floor",)*obj_num+sweep_goal]
if sweep_idx < sweep_num-1:
    for i_s in range(obj_num):
        obj_goal = ["floor"]*obj_num
        obj_goal[i_s] = "grip1"
        goal_nodes += [tuple(obj_goal)+sweep_goal]
with gtimer.block("plan{}".format(sweep_idx), stack=True):
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=50, multiprocess=True, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
snode_schedule = mix_schedule(mplan, snode_schedule_safe)
from_state = snode_schedule[-1].state
snode_schedule_all.append(snode_schedule)
with gtimer.block("exe{}".format(sweep_idx), stack=True):
    ppline.execute_schedule(snode_schedule, mode_switcher=msfl)

In [ ]:
sweep_idx = 2
sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
goal_nodes = [("floor",)*obj_num+sweep_goal]
if sweep_idx < sweep_num-1:
    for i_s in range(obj_num):
        obj_goal = ["floor"]*obj_num
        obj_goal[i_s] = "grip1"
        goal_nodes += [tuple(obj_goal)+sweep_goal]
with gtimer.block("plan{}".format(sweep_idx), stack=True):
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=100, multiprocess=True, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
snode_schedule = mix_schedule(mplan, snode_schedule_safe)
from_state = snode_schedule[-1].state
snode_schedule_all.append(snode_schedule)
with gtimer.block("exe{}".format(sweep_idx), stack=True):
    ppline.execute_schedule(snode_schedule, mode_switcher=msfl)

In [ ]:
sweep_idx = 3
sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
goal_nodes = [("floor",)*obj_num+sweep_goal]
if sweep_idx < sweep_num-1:
    for i_s in range(obj_num):
        obj_goal = ["floor"]*obj_num
        obj_goal[i_s] = "grip1"
        goal_nodes += [tuple(obj_goal)+sweep_goal]
with gtimer.block("plan{}".format(sweep_idx), stack=True):
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=50, multiprocess=True, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
snode_schedule = mix_schedule(mplan, snode_schedule_safe)
from_state = snode_schedule[-1].state
snode_schedule_all.append(snode_schedule)
with gtimer.block("exe{}".format(sweep_idx), stack=True):
    ppline.execute_schedule(snode_schedule, mode_switcher=msfl)

In [ ]:
sweep_idx = 4
sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(sweep_num)])
#     sweep_goal = tuple([int(i_s<=sweep_idx)*2 for i_s in range(2)])+(0,)
goal_nodes = [("floor",)*obj_num+sweep_goal]
if sweep_idx < sweep_num-1:
    for i_s in range(obj_num):
        obj_goal = ["floor"]*obj_num
        obj_goal[i_s] = "grip1"
        goal_nodes += [tuple(obj_goal)+sweep_goal]
with gtimer.block("plan{}".format(sweep_idx), stack=True):
    ppline.search(from_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
                  timeout_loop=50, multiprocess=True, timeout=5,
                  plannerconfig=PlannerConfig.RRTConnectkConfigDefault, cs_type=ConstrainedSpaceType.TANGENTBUNDLE, post_projection=True)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])
snode_schedule_simple = simplify_schedule(pscene, snode_schedule)
snode_schedule_safe = calculate_safe_schedule(pscene, snode_schedule_simple, 0.5, 1)
snode_schedule = mix_schedule(mplan, snode_schedule_safe)
from_state = snode_schedule[-1].state
snode_schedule_all.append(snode_schedule)
with gtimer.block("exe{}".format(sweep_idx), stack=True):
    ppline.execute_schedule(snode_schedule, mode_switcher=msfl)

In [ ]:
print(gtimer)

In [ ]:
# for i_ss, snode_schedule in enumerate(snode_schedule_all):
#     for i_s, snode in enumerate(snode_schedule):
#         if snode.traj is not None:
#             save_json("data/traj_{}_{}.json".format(i_ss, i_s), snode.traj[:,:6])

## mix full schedule

In [ ]:
gtimer.reset()
snode_schedule_cat = [snode_schedule_all[0][0]]
for snode_schedule in snode_schedule_all:
    snode_schedule_cat += snode_schedule[1:]

with gtimer.block("mix_schedule_safe"):
    safe_mixed = mix_schedule(mplan, snode_schedule_cat)

print(gtimer)

### play schedule

In [ ]:
ppline.play_schedule(safe_mixed, period=0.001)

In [ ]:
ppline.execute_schedule(safe_mixed, mode_switcher=ModeSwitcher(pscene))